In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import glob
import pandas as pd
import os
import numpy as np
import pickle
import sys

In [3]:
import dataset_confs

In [4]:
from DatasetManager import DatasetManager

Using TensorFlow backend.


# Dataset stats

In [35]:
home_dir = ""

In [37]:
dt_stats = []
datasets = ["bpic2011_f%s"%formula for formula in range(1,5)] + ["bpic2015_%s_f2"%(municipality) for municipality in range(1,6)] + ["production",
                                                                                                                                    "sepsis_cases_1", "sepsis_cases_2", "sepsis_cases_4",
                                                                                                                                    "bpic2012_accepted", "bpic2012_declined", "bpic2012_cancelled",
                                                                                                                                    "bpic2017_accepted", "bpic2017_refused", "bpic2017_cancelled",
                                                                                                                                    "traffic_fines_1",]
#datasets = ["unemployment"]
print("\\toprule  &  & min  & med  & max      & \\# distinct & \\# static  & \\# event  & \\# case  & \\# dynamic \\\\ ")
print(" dataset & \\# cases &  length &  length & length &   activities &    variables &   variables &  cat levels & cat levels\\\\ \\midrule")
for dataset_name in datasets:
    dataset_manager = DatasetManager(dataset_name)
    
    case_id_col = dataset_confs.case_id_col[dataset_name]
    activity_col = dataset_confs.activity_col[dataset_name]
    timestamp_col = dataset_confs.timestamp_col[dataset_name]
    label_col = dataset_confs.label_col[dataset_name]
    pos_label = dataset_confs.pos_label[dataset_name]

    dynamic_cat_cols = dataset_confs.dynamic_cat_cols[dataset_name]
    static_cat_cols = dataset_confs.static_cat_cols[dataset_name]
    dynamic_num_cols = dataset_confs.dynamic_num_cols[dataset_name]
    static_num_cols = dataset_confs.static_num_cols[dataset_name]

    data_filepath = os.path.join(home_dir, dataset_confs.filename[dataset_name])

    # specify data types
    dtypes = {col:"object" for col in dynamic_cat_cols+static_cat_cols+[case_id_col, label_col, timestamp_col]}
    for col in dynamic_num_cols + static_num_cols:
        dtypes[col] = "float"
            
    data = pd.read_csv(data_filepath, sep=";", dtype=dtypes)
    sizes = data.groupby(case_id_col).size()

    class_freqs = data.groupby(case_id_col).first()[label_col].value_counts()
    
    if "traffic_fines" in dataset_name:
        max_prefix_length = 10
    elif "bpic2017" in dataset_name:
        max_prefix_length = min(20, dataset_manager.get_pos_case_length_quantile(data, 0.90))
    else:
        max_prefix_length = min(40, dataset_manager.get_pos_case_length_quantile(data, 0.90))
    
    n_trace_variants = len(data.sort_values(timestamp_col, kind="mergesort").groupby(case_id_col).head(max_prefix_length).groupby(case_id_col)[activity_col].apply(lambda x: "__".join(list(x))).unique())
    
    n_static_cat_levels = 0
    n_dynamic_cat_levels = 0
    for col in dynamic_cat_cols:
        n_dynamic_cat_levels += len(data[col].unique())
    for col in static_cat_cols:
        n_static_cat_levels += len(data[col].unique())
    
    dataset_name = dataset_name.replace("_", "\\_").replace("bpic2011\_f", "bpic2011\_").replace("\_f2", "").replace("activity", "1").replace("followup", "2").replace("cases\_", "").replace("billing\_", "").replace("accepted", "1").replace("declined", "2").replace("refused", "2").replace("cancelled", "3").replace("\_fines\_1", "").replace("sepsis\_4", "sepsis\_3").replace("hospital\_2", "hospital\_1").replace("hospital\_3", "hospital\_2")
    print("%s & %s & %s & %s  & %s & %s & %s & %s & %s & %s \\\\"%(dataset_name, len(data[case_id_col].unique()), sizes.min(), sizes.quantile(0.50), sizes.max(), len(data[activity_col].unique()), len(static_cat_cols) + len(static_num_cols), len(dynamic_cat_cols) + len(dynamic_num_cols),
                                                                      n_static_cat_levels, n_dynamic_cat_levels))
    
    record = (dataset_name, len(data[case_id_col].unique()), sizes.min(), sizes.quantile(0.50), sizes.max(), len(data[activity_col].unique()), len(static_cat_cols) + len(static_num_cols), len(dynamic_cat_cols) + len(dynamic_num_cols),
                                                                      n_static_cat_levels, n_dynamic_cat_levels)
    dt_stats.append(record)
    
print("\\bottomrule")

\toprule  &  & min  & med  & max      & \# distinct & \# static  & \# dynamic  & \# static  & \# dynamic \\ 
 dataset & \# traces &  length &  length & length &   activities &    attr-s &   attr-s &  cat levels & cat levels\\ \midrule
bpic2011\_1 & 1140 & 1 & 25.0  & 1814 & 193 & 6 & 14 & 961 & 290 \\
bpic2011\_2 & 1140 & 1 & 54.5  & 1814 & 251 & 6 & 14 & 994 & 370 \\
bpic2011\_3 & 1121 & 1 & 21.0  & 1368 & 190 & 6 & 14 & 886 & 283 \\
bpic2011\_4 & 1140 & 1 & 44.0  & 1432 & 231 & 6 & 14 & 993 & 338 \\
bpic2015\_1 & 694 & 2 & 42.5  & 101 & 380 & 17 & 12 & 17 & 432 \\
bpic2015\_2 & 753 & 1 & 55.0  & 132 & 396 & 17 & 12 & 7 & 429 \\
bpic2015\_3 & 1325 & 3 & 42.0  & 124 & 380 & 18 & 12 & 17 & 428 \\
bpic2015\_4 & 577 & 1 & 42.0  & 82 & 319 & 15 & 12 & 9 & 347 \\
bpic2015\_5 & 1051 & 5 & 50.0  & 134 & 376 & 18 & 12 & 8 & 419 \\
production & 220 & 1 & 9.0  & 78 & 26 & 3 & 15 & 37 & 79 \\
sepsis\_1 & 754 & 5 & 14.0  & 185 & 14 & 24 & 13 & 195 & 38 \\
sepsis\_2 & 782 & 4 & 13.0  & 60 & 15 & 24

# AUC

In [13]:
!pwd

/home/kamel/utcs/master_courses/4th/thesis/new_repo/predictive-monitoring-thesis


In [74]:
data_cat=pd.read_csv('./../../results_all/catboost/code/final_data/auc/data_catboost.csv', sep = ';')
data_cat.head()

data_wavelet=pd.read_csv('./../../results_all/catboost/code/final_data/auc/data_wavelet.csv', sep = ';')
data_wavelet.head()

data_inter=pd.read_csv('./../../results_all/catboost/code/final_data/auc/Inter_data_catboost.csv', sep = ';')
data_inter.head()

,dataset,method,cls,nr_events,metric,score
0,production,prefix_agg,xgboost,1,auc,0.603571
1,production,prefix_agg,xgboost,1,prec,0.393939
2,production,prefix_agg,xgboost,1,rec,0.928571
3,production,prefix_agg,xgboost,1,fscore,0.553191
4,production,prefix_agg,xgboost,2,auc,0.608466


In [75]:
set(data['dataset'])

{'bpic2011_1',
 'bpic2011_2',
 'bpic2011_3',
 'bpic2011_4',
 'bpic2012_a',
 'bpic2012_c',
 'bpic2012_d',
 'bpic2015_1',
 'bpic2015_2',
 'bpic2015_3',
 'bpic2015_4',
 'bpic2015_5',
 'bpic2017_a',
 'bpic2017_c',
 'bpic2017_r',
 'production',
 'sepsis_1',
 'sepsis_2',
 'sepsis_3'}

In [76]:
#data = pd.read_csv('./../../results_all/catboost/code/final_data/auc/data_wavelet_NODUPLICATE.csv', sep = ';')
data = data_inter

data.dataset = data.dataset.str.replace("sepsis_1", "sepsis_cases_1")
data.dataset = data.dataset.str.replace("sepsis_2", "sepsis_cases_2")
data.dataset = data.dataset.str.replace("sepsis_3", "sepsis_cases_4")

data.dataset = data.dataset.str.replace("bpic2011_4", "bpic2011_f4")
data.dataset = data.dataset.str.replace("bpic2011_3", "bpic2011_f3")
data.dataset = data.dataset.str.replace("bpic2011_2", "bpic2011_f2")
data.dataset = data.dataset.str.replace("bpic2011_1", "bpic2011_f1")

data.dataset = data.dataset.str.replace("bpic2015_1", "bpic2015_1_f2")
data.dataset = data.dataset.str.replace("bpic2015_2", "bpic2015_2_f2")
data.dataset = data.dataset.str.replace("bpic2015_3", "bpic2015_3_f2")
data.dataset = data.dataset.str.replace("bpic2015_4", "bpic2015_4_f2")
data.dataset = data.dataset.str.replace("bpic2015_5", "bpic2015_5_f2")

data.dataset = data.dataset.str.replace("bpic2012_A", "bpic2012_accepted")
data.dataset = data.dataset.str.replace("bpic2012_C", "bpic2012_cancelled")
data.dataset = data.dataset.str.replace("bpic2012_D", "bpic2012_declined")

data.dataset = data.dataset.str.replace("bpic2017_R", "bpic2017_refused")
data.dataset = data.dataset.str.replace("bpic2017_A", "bpic2017_accepted")
data.dataset = data.dataset.str.replace("bpic2017_C", "bpic2017_cancelled")


data.dataset = data.dataset.str.replace("traffic", "traffic_fines_1")


In [78]:
   
with open("n_test_cases.pickle", "rb") as fin:
    test_cases_dict = pickle.load(fin)
    
    
df_test_cases = pd.DataFrame(test_cases_dict)
df_test_cases = df_test_cases.stack().reset_index()
df_test_cases.columns = ["nr_events", "dataset", "n_cases"]
df_test_cases
data = pd.merge(data, df_test_cases, on=["dataset", "nr_events"])
data

,dataset,method,cls,nr_events,metric,score,n_cases
0,production,prefix_agg,xgboost,1,auc,0.603571,44.0
1,production,prefix_agg,xgboost,1,prec,0.393939,44.0
2,production,prefix_agg,xgboost,1,rec,0.928571,44.0
3,production,prefix_agg,xgboost,1,fscore,0.553191,44.0
4,production,single_agg,logit,1,auc,0.528571,44.0
...,...,...,...,...,...,...,...
49091,bpic2015_3_f2,prefix_laststate,svm,1,fscore,0.000000,265.0
49092,bpic2015_3_f2,state_agg,rf,1,auc,0.659882,265.0
49093,bpic2015_3_f2,state_agg,rf,1,prec,0.571429,265.0
49094,bpic2015_3_f2,state_agg,rf,1,rec,0.059701,265.0


In [79]:
data.dataset = data.dataset.str.replace("sepsis_cases_1", "sepsis_1")
data.dataset = data.dataset.str.replace("sepsis_cases_2", "sepsis_2")
data.dataset = data.dataset.str.replace("sepsis_cases_4", "sepsis_3")

data.dataset = data.dataset.str.replace("bpic2011_f4", "bpic2011_4")
data.dataset = data.dataset.str.replace("bpic2011_f3", "bpic2011_3")
data.dataset = data.dataset.str.replace("bpic2011_f2", "bpic2011_2")
data.dataset = data.dataset.str.replace("bpic2011_f1", "bpic2011_1")

data.dataset = data.dataset.str.replace("bpic2015_1_f2", "bpic2015_1")
data.dataset = data.dataset.str.replace("bpic2015_2_f2", "bpic2015_2")
data.dataset = data.dataset.str.replace("bpic2015_3_f2", "bpic2015_3")
data.dataset = data.dataset.str.replace("bpic2015_4_f2", "bpic2015_4")
data.dataset = data.dataset.str.replace("bpic2015_5_f2", "bpic2015_5")

data.dataset = data.dataset.str.replace("bpic2012_accepted", "bpic2012_a")
data.dataset = data.dataset.str.replace("bpic2012_cancelled", "bpic2012_c")
data.dataset = data.dataset.str.replace("bpic2012_declined", "bpic2012_d")

data.dataset = data.dataset.str.replace("bpic2017_refused", "bpic2017_r")
data.dataset = data.dataset.str.replace("bpic2017_accepted", "bpic2017_a")
data.dataset = data.dataset.str.replace("bpic2017_cancelled", "bpic2017_c")

data.dataset = data.dataset.str.replace("hospital_billing_2", "hospital_2")
data.dataset = data.dataset.str.replace("hospital_billing_3", "hospital_3")

data.dataset = data.dataset.str.replace("traffic_fines_1", "traffic")
#data.method = data.method.str.replace("prefix_index_5", "prefix_index")




In [83]:
round_prec = 2
tmp_auc = data[(data.metric=="auc")]
tmp_auc.fillna(0, inplace=True)
all_mean_aucs = tmp_auc.groupby(["dataset", "method", "cls"]).apply(lambda group: np.average(group["score"], 
                                                                           weights=group["n_cases"])).reset_index()
all_mean_aucs.columns = ["dataset", "method", "cls", "score"]
best_mean_aucs = all_mean_aucs.groupby(["dataset"])["score"].max().round(round_prec)


bucket_encs = list(set(data.method))
methods = ["%s" % (bucket_enc) for bucket_enc in bucket_encs]
clfs = ["catboost","rf", "xgboost", "logit","svm"]
#cls_methods = ["catboost","rf", "xgboost", "logit","svm"]
tmp2_auc = tmp_auc.groupby("method")


In [84]:
round_prec = 2
tmp_fsc = data[(data.metric=="fscore")]
tmp_fsc.fillna(0, inplace=True)
all_mean_fsc = tmp_fsc.groupby(["dataset", "method", "cls"]).apply(lambda group: np.average(group["score"], 
                                                                           weights=group["n_cases"])).reset_index()
all_mean_fsc.columns = ["dataset", "method", "cls", "score"]
best_mean_fsc = all_mean_fsc.groupby(["dataset"])["score"].max().round(round_prec)


bucket_encs = list(set(data.method))
methods = ["%s" % (bucket_enc) for bucket_enc in bucket_encs]
clfs = ["catboost","rf", "xgboost", "logit","svm"]
#cls_methods = ["catboost","rf", "xgboost", "logit","svm"]
tmp2_fsc = tmp_fsc.groupby("method")


## Run this many times for many tables 0:6 datasets

In [98]:

i=0
for k in range(4):    
    datasets = list(set(data['dataset']))[i:i+6]
    #print(datasets)
    i+=6
    
    # change this line for each classifiere
    cls_methods=['svm']
    #datasets = list(set(data['dataset']))[0:6]
    print("\\toprule")
    print(" & ".join([""] + ["\\multicolumn{5}{c}{%s}" % cls for cls in cls_methods]))
    print("\\\\")
    print(" & ".join([""] + ["%s"%dataset.replace("_", "\\_") for dataset in datasets] * len(cls_methods)))
    print("\\\\ \\midrule")
    for method in methods:
        elems_to_print = [method.replace("_", "\\_")]
        if method not in tmp2_auc.groups:
            continue
        tmp_method_auc = tmp2_auc.get_group(method)
        
        if method not in tmp2_fsc.groups:
            continue
        tmp_method_fsc = tmp2_fsc.get_group(method)

        tmp_method_cls_auc = tmp_method_auc.groupby("cls")
        tmp_method_cls_fsc = tmp_method_fsc.groupby("cls")
        for cls in cls_methods:
            #print(cls)
            tmp_cls_auc = tmp_method_cls_auc.get_group(cls)            
            tmp_method_grouped_auc = tmp_cls_auc.groupby("dataset")
            
            tmp_cls_fsc = tmp_method_cls_fsc.get_group(cls)            
            tmp_method_grouped_fsc = tmp_cls_fsc.groupby("dataset")
            
            
            #print(tmp_method_grouped.head())
            for dataset_name in datasets:  
                #print(dataset_name)
                if dataset_name not in tmp_method_grouped_auc.groups:
                    elems_to_print.append("$-$ ")
                    continue
                tmp_dataset_auc = tmp_method_grouped_auc.get_group(dataset_name)
                tmp_dataset_fsc = tmp_method_grouped_fsc.get_group(dataset_name)


                mean_value_auc = np.average(tmp_dataset_auc["score"], weights=tmp_dataset_auc["n_cases"])
                #print(mean_value)
                variance_auc = np.average((tmp_dataset_auc["score"]-mean_value_auc)**2, weights=tmp_dataset_auc["n_cases"])
                std_value_auc = round(np.sqrt(variance_auc), round_prec)
                mean_value_auc = round(mean_value_auc, round_prec)
                
                mean_value_fsc = np.average(tmp_dataset_fsc["score"], weights=tmp_dataset_fsc["n_cases"])
                #print(mean_value)
                variance_fsc = np.average((tmp_dataset_fsc["score"]-mean_value_fsc)**2, weights=tmp_dataset_fsc["n_cases"])
                std_value_fsc = round(np.sqrt(variance_fsc), round_prec)
                mean_value_fsc = round(mean_value_fsc, round_prec)

                current_str = ""
                if mean_value_auc == best_mean_aucs[dataset_name]:
                    current_str = "$\\mathbf{%s}$ $\\mathbf{(%s)}$ "%(mean_value_auc, mean_value_fsc)
                else:
                    current_str = "$%s$ ${(%s)}$" % (mean_value_auc, mean_value_fsc)

                elems_to_print.append(current_str)

        print("%s \\\\"%(" & ".join(elems_to_print)))

    print("\\bottomrule")


\toprule
 & \multicolumn{5}{c}{svm}
\\
 & bpic2015\_2 & bpic2015\_4 & bpic2011\_4 & bpic2017\_r & bpic2012\_d & bpic2012\_a
\\ \midrule
cluster\_laststate & $0.56$ ${(0.0)}$ & $0.58$ ${(0.0)}$ & $0.51$ ${(0.0)}$ & $0.55$ ${(0.0)}$ & $0.5$ ${(0.0)}$ & $0.66$ ${(0.32)}$ \\
cluster\_agg & $0.67$ ${(0.46)}$ & $0.56$ ${(0.0)}$ & $0.73$ ${(0.07)}$ & $0.5$ ${(0.0)}$ & $0.53$ ${(0.0)}$ & $0.49$ ${(0.0)}$ \\
single\_agg & $0.5$ ${(0.0)}$ & $0.5$ ${(0.0)}$ & $0.5$ ${(0.0)}$ & $0.5$ ${(0.0)}$ & $0.5$ ${(0.0)}$ & $0.51$ ${(0.71)}$ \\
prefix\_agg & $0.5$ ${(0.0)}$ & $0.5$ ${(0.0)}$ & $0.88$ ${(0.0)}$ & $0.5$ ${(0.0)}$ & $0.51$ ${(0.01)}$ & $0.5$ ${(0.3)}$ \\
prefix\_laststate & $0.5$ ${(0.0)}$ & $0.5$ ${(0.0)}$ & $0.84$ ${(0.0)}$ & $0.56$ ${(0.0)}$ & $0.5$ ${(0.0)}$ & $0.5$ ${(0.0)}$ \\
single\_laststate & $0.5$ ${(0.0)}$ & $0.53$ ${(0.0)}$ & $0.84$ ${(0.44)}$ & $-$  & $0.54$ ${(0.08)}$ & $0.5$ ${(0.0)}$ \\
prefix\_index & $0.68$ ${(0.43)}$ & $0.53$ ${(0.0)}$ & $0.88$ ${(0.71)}$ & $0.5$ ${(0.0)}$ &

# Execution time

In [104]:
data_cat=pd.read_csv('./../../results_all/catboost/code/final_data/time/Inter_data_time_catboost.csv', sep = ';')
data = data_cat
clfs = ['xgboost']
for cls in clfs:
    print(f"\n\n\n\ncls: {cls}\n\n\n\n")
    cls = cls
    data_cls = data[data.cls==cls]
    best_offline_times = data_cls[data_cls.metric=="offline_total_avg"].groupby(["dataset"])["score"].min().round(2)
    best_online_times = data_cls[data_cls.metric=="online_event_avg"].groupby(["dataset"])["score"].min().round(2)
    
    bucket_encs = list(set(data.bucket_enc))
    methods = ["%s" % (bucket_enc) for bucket_enc in bucket_encs]
    cls_methods = cls
 
    # 6 by 6 for tables
    i = 0
    for k in range(7):
        datasets =list(set(data['dataset']))[i:i+3]
        i+=3

        
        print("\\toprule")
        print("%s \\\\ \\cmidrule(lr){2-3} \\cmidrule(lr){4-5} \\cmidrule(lr){6-7}"%("".join([" & \multicolumn{2}{c}{{\\bfseries %s}}"%(dataset.replace("_", "\\_")) for dataset in datasets])))
        print("method " + " & training\\_total (s) & prediction\\_avg (ms)" * len(datasets) + " \\\\ \midrule")

        for bucket_enc in bucket_encs:
        
            elems_to_print = [bucket_enc.replace("_", "\\_")]
            for dataset in datasets:

                dt_sub = data_cls[(data_cls.dataset==dataset) & (data_cls.bucket_enc==bucket_enc) ]
                if len(dt_sub) > 0:
                    offline_mean = round(dt_sub[dt_sub.metric=="offline_total_avg"].score.iloc[0], 2)
                    offline_std = round(dt_sub[dt_sub.metric=="offline_total_std"].score.iloc[0], 2)
                    online_mean = round(dt_sub[dt_sub.metric=="online_event_avg"].score.iloc[0], 2)
                    online_std = round(dt_sub[dt_sub.metric=="online_event_std"].score.iloc[0], 2)

                    if offline_mean == best_offline_times[dataset]:
                        elems_to_print.append("$\\mathbf{%s \\pm %s}$" % (offline_mean, offline_std))
                    else:
                        elems_to_print.append("$%s \\pm %s$" % (offline_mean, offline_std))

                    if online_mean == best_online_times[dataset]:
                        elems_to_print.append("$\\mathbf{%s \\pm %s}$" % (online_mean, online_std))
                    else:
                        elems_to_print.append("$%s \\pm %s$" % (online_mean, online_std))

                else:
                    elems_to_print.extend(["-", "-"])

            print("%s \\\\ "%(" & ".join(elems_to_print)))
        print("\\bottomrule")

    #break
    #datasets = list(set(data['dataset']))[0:3]
    





cls: xgboost




\toprule
 & \multicolumn{2}{c}{{\bfseries bpic2015\_2}} & \multicolumn{2}{c}{{\bfseries bpic2012\_C}} & \multicolumn{2}{c}{{\bfseries bpic2015\_4}} \\ \cmidrule(lr){2-3} \cmidrule(lr){4-5} \cmidrule(lr){6-7}
method  & training\_total (s) & prediction\_avg (ms) & training\_total (s) & prediction\_avg (ms) & training\_total (s) & prediction\_avg (ms) \\ \midrule
cluster\_laststate & $4.88 \pm 0.0$ & $0.01 \pm 0.01$ & $60.94 \pm 0.0$ & $0.01 \pm 0.01$ & $3.64 \pm 0.0$ & $0.01 \pm 0.01$ \\ 
cluster\_agg & $5.41 \pm 0.0$ & $0.01 \pm 0.02$ & $75.17 \pm 0.0$ & $0.01 \pm 0.01$ & $4.77 \pm 0.0$ & $0.01 \pm 0.02$ \\ 
single\_agg & $18.77 \pm 0.0$ & $0.01 \pm 0.01$ & $106.34 \pm 0.0$ & $\mathbf{0.0 \pm 0.01}$ & $7.03 \pm 0.0$ & $0.01 \pm 0.01$ \\ 
prefix\_agg & $0.85 \pm 0.0$ & $\mathbf{0.0 \pm 0.0}$ & $2.5 \pm 0.0$ & $\mathbf{0.0 \pm 0.0}$ & $0.74 \pm 0.0$ & $\mathbf{0.0 \pm 0.0}$ \\ 
prefix\_laststate & $0.86 \pm 0.0$ & $\mathbf{0.0 \pm 0.0}$ & $2.55 \pm 0.0$ & $\mathbf{0.0

prefix\_laststate & $0.76 \pm 0.0$ & $\mathbf{0.0 \pm 0.0}$ & $0.12 \pm 0.0$ & $\mathbf{0.0 \pm 0.0}$ & $0.81 \pm 0.0$ & $\mathbf{0.0 \pm 0.0}$ \\ 
single\_laststate & $6.36 \pm 0.0$ & $0.01 \pm 0.01$ & $0.38 \pm 0.0$ & $0.01 \pm 0.01$ & $20.77 \pm 0.0$ & $0.02 \pm 0.03$ \\ 
prefix\_index & $\mathbf{0.11 \pm 0.0}$ & $\mathbf{0.0 \pm 0.0}$ & $\mathbf{0.04 \pm 0.0}$ & $\mathbf{0.0 \pm 0.0}$ & $\mathbf{0.18 \pm 0.0}$ & $\mathbf{0.0 \pm 0.0}$ \\ 
state\_agg & $11.19 \pm 0.0$ & $0.01 \pm 0.02$ & $0.72 \pm 0.0$ & $0.01 \pm 0.01$ & $15.51 \pm 0.0$ & $0.02 \pm 0.03$ \\ 
state\_laststate & $8.72 \pm 0.0$ & $0.01 \pm 0.02$ & $0.54 \pm 0.0$ & $0.01 \pm 0.01$ & $17.31 \pm 0.0$ & $0.02 \pm 0.03$ \\ 
\bottomrule
\toprule
 & \multicolumn{2}{c}{{\bfseries bpic2017\_R}} \\ \cmidrule(lr){2-3} \cmidrule(lr){4-5} \cmidrule(lr){6-7}
method  & training\_total (s) & prediction\_avg (ms) \\ \midrule
cluster\_laststate & $377.35 \pm 0.0$ & $0.01 \pm 0.01$ \\ 
cluster\_agg & $986.18 \pm 0.0$ & $0.01 \pm 0.01$ \